In [1]:
import pandas as pd
import os
import shutil
import librosa
import numpy as np
import soundfile as sf
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
import random
import tensorflow_io as tfio
import preprocessing as pr
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import argparse as ap

2023-01-16 12:40:57.807802: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-16 12:40:57.873689: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-16 12:40:57.876835: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-16 12:40:57.876843: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

31


2023-01-16 12:40:58.635107: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-16 12:40:58.635279: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-16 12:40:58.635308: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-01-16 12:40:58.635331: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-01-16 12:40:58.635354: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [2]:
!echo "Be sure to have tensorboard, this code will be commented in final release"
#!tensorboard --logdir ../../datasets/dsl_data/tensorboard_data/ &

Be sure to have tensorboard, this code will be commented in final release


 # Preprocessing for Train dataset files

In [3]:
# process_file better to be implemented here with a boolean value that checks if i am processing train_dataset or eval file
def process_file(file_path, flag):
    file_path_exists = df[df["path"] == file_path].shape[0] > 0 #flag
    if file_path_exists:
        new_sr=16000
        # identifier care
        identifier = df.loc[df["path"] == file_path, "Id"].values[0]
        identifier = str(int(identifier))
        # label constructor
        label = ""
        if flag == 1:
            label  += "_"
            action  = df.loc[df["path"] == file_path, "action"].values[0]
            object  = df.loc[df["path"] == file_path, "object"].values[0]
            label  += action + object
        # If no label available, code will just go on
        new_file_path = os.path.join(new_folder_path, identifier + label + '.wav')
        y, sr = librosa.load('../../datasets/'+file_path)
        y_truncated = librosa.effects.trim(y, top_db=50, frame_length=2048, hop_length=512, ref=np.max)[0]
        y_truncated = librosa.resample(y_truncated, orig_sr=sr, target_sr=new_sr)
        y_truncated = y_truncated[:int(4*new_sr)] #if longer
        target_length = 4 * new_sr
        y_truncated = librosa.util.fix_length(data=y_truncated, size=target_length) # padding, if shorter
        sf.write(new_file_path, y_truncated, new_sr, 'PCM_16')

In [4]:
df = pd.read_csv('../../datasets/dsl_data/development.csv', sep=',')
new_folder_path = '../../datasets/dsl_data/Train_Dataset_Truncated/'

folder_path = '../../datasets/dsl_data/'

if not os.path.isdir(new_folder_path):
    os.makedirs(new_folder_path) # hoping to have write permissions set
if not os.listdir(new_folder_path):
    with ThreadPoolExecutor() as executor: # who is your single threaddy?
        for dirpath, dirnames, filenames in os.walk(folder_path):
            dirpath = dirpath.replace("\\", "/")
            dirpath = dirpath[dirpath.index("/")+1:]
            dirpath = dirpath[dirpath.index("/")+1:]
            dirpath = dirpath[dirpath.index("/")+1:]
            for filename in filenames:
                file_path = os.path.join(dirpath, filename)
                file_path = file_path.replace("\\", "/")
                executor.submit(process_file, file_path, 1)
# print(df)
print("Execution ended")


Execution ended


 # Preprocessing for Evaluation dataset files

In [5]:
df = pd.read_csv('../../datasets/dsl_data/evaluation.csv', sep=',')
new_folder_path = '../../datasets/dsl_data/Test_Dataset_Truncated/'
folder_path = '../../datasets/dsl_data/'

if not os.path.isdir(new_folder_path):
    os.makedirs(new_folder_path)

if not os.listdir(new_folder_path):
    with ThreadPoolExecutor() as executor:
        for dirpath, dirnames, filenames in os.walk(folder_path):
            dirpath = dirpath.replace("\\", "/")
            dirpath = dirpath[dirpath.index("/")+1:]
            dirpath = dirpath[dirpath.index("/")+1:]
            dirpath = dirpath[dirpath.index("/")+1:]
            for filename in filenames:
                file_path = os.path.join(dirpath, filename)
                file_path = file_path.replace("\\", "/")
                executor.submit(process_file, file_path, 0)

print("Execution ended")

Execution ended


# Auto - updating labels

In [6]:
df = pd.read_csv('../../datasets/dsl_data/development.csv', sep=',')
df['labels'] = df['action'].astype(str) + df['object'].astype(str)
distinct_values = df['labels'].unique()

result = 'LABELS = ['
for value in distinct_values:
    result += "'" + str(value) + "', "

result = result[:-2] + ']\n' # lazy workaround, the last label has a comma that is bad.. this is also bad.

with open("preprocessing.py", "r") as file:
    lines = file.readlines()

# Update the 4th line
lines[3] = result
lines[4] = "# This is the file genarated that has the Labels that i must use for training\n"

with open("preprocessing.py", "w") as file:
    file.writelines(lines)

# Model creation and fitting

In [7]:
parser = ap.ArgumentParser()

parser.add_argument('--batch_size', default=pr.TRAINING_ARGS['batch_size'], type=int, help="Choosing batch size default is 32")
parser.add_argument('--initial_learning_rate', default=pr.TRAINING_ARGS['initial_learning_rate'], type=float, help="Choosing initial_learning_rate")
parser.add_argument('--end_learning_rate', default=pr.TRAINING_ARGS['end_learning_rate'], type=float, help="Choosing end_learning_rate")
parser.add_argument('--epochs', default=pr.TRAINING_ARGS['epochs'], type=int, help="Choosing epochs")
parser.add_argument('--test_percentage', default=0.2, type=float, help="Choosing test_percentage")
parser.add_argument('--pruning_initial_step', default=0.2, type=float, help="Choosing pruning_initial_step")
parser.add_argument('--initial_sparsity', default=0.40, type=float, help="Choosing initial_sparsity")
parser.add_argument('--alpha', default=pr.alpha, type=float, help="Choosing alpha")

parser.add_argument('--eval_percentage', default=0.15, type=float, help="Choosing eval_percentage")

_StoreAction(option_strings=['--eval_percentage'], dest='eval_percentage', nargs=None, const=None, default=0.15, type=<class 'float'>, choices=None, required=False, help='Choosing eval_percentage', metavar=None)

Parser arguments

In [8]:
args = parser.parse_args(['--epochs','60','--batch_size','64','--pruning_initial_step','0.2','--initial_learning_rate','0.03','--end_learning_rate','0.001'])
# args = parser.parse_args()

## This part of the code exist to manage all the folders
## Please be careful, if the directories tree is not respected, the code will not work properly

In [9]:
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

In [10]:
# Useful to save tensorboard data
log_dir_tensorboard = '../../datasets/dsl_data/tensorboard_data/'
if not os.path.isdir(log_dir_tensorboard):
    os.makedirs(log_dir_tensorboard)
runs = [int(d.split('_')[1]) for d in os.listdir(log_dir_tensorboard) if 'run_' in d]
tb_run = max(runs) + 1 if runs else 0

In [11]:
# Folder creation
train_ds_location   = '../../datasets/dsl_data/Train_Dataset_Truncated/'
log_dir_model       = '../../datasets/dsl_data/models/'
model_name          = 'run_{}_batch_size_{}_pruning_initial_step_{}_initial_learning_rate_{}_end_learning_rate_{}_test_percentage_{}_pruning_initial_step_{}_initial_sparsity_{}_alpha_{}'.format(tb_run,args.batch_size,args.pruning_initial_step,args.initial_learning_rate,args.end_learning_rate,args.test_percentage,args.pruning_initial_step,args.initial_sparsity,args.alpha)
checkpoint_path     = '../../datasets/dsl_data/checkpoints/' + model_name+'.ckpt'

In [12]:
# If folders to not exist -> create them
# This code will not check for the dataset folders, the code above must be executed
if not os.path.isdir(log_dir_model):
    os.makedirs(log_dir_model)
if not os.path.isdir(checkpoint_path):
    os.makedirs(checkpoint_path)

Obtaining Test data from train data, using shuffle and avoiding retaking same data on different runs

In [13]:
file_paths = []

for filename in os.listdir(train_ds_location):
    file_path = os.path.join(train_ds_location, filename)
    file_paths.append(file_path)
random.shuffle(file_paths)
num_test_files = int(len(file_paths) * args.test_percentage)
num_eval_files = int(len(file_paths) * args.eval_percentage)

# it is shuffled, so i can do this
test_paths     = file_paths[:num_test_files]                 # from 0 to num_test_files
train_paths    = file_paths[num_test_files:-num_eval_files]  # from num_test_files to end-num_eval_files
eval_paths     = file_paths[-num_eval_files:]                # until the end


Preprocessing data and model creation

In [14]:
train_ds       = tf.data.Dataset.list_files(train_paths)
val_ds         = tf.data.Dataset.list_files(eval_paths)
test_ds        = tf.data.Dataset.list_files(test_paths)

train_ds       = train_ds.map(pr.preprocess).batch(args.batch_size).cache()
val_ds         = val_ds.map(pr.preprocess).batch(args.batch_size)
test_ds        = test_ds.map(pr.preprocess).batch(args.batch_size)

for example_batch, example_labels in train_ds.take(1):
  print('Batch Shape:', example_batch.shape)
  print('Data Shape:', example_batch.shape[1:])
  print('Labels:', example_labels)

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
begin_step          = int(len(train_ds) * args.epochs * args.pruning_initial_step)
end_step            = int(len(train_ds) * args.epochs)
pruning_params      = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=args.initial_sparsity,
        final_sparsity=pr.final_sparsity,
        begin_step=begin_step,
        end_step=end_step
    )
}
custom_objects      = {'PruneLowMagnitude': prune_low_magnitude}

#model_name          = 'model_'+str(args.batch_size)+'_'+str(args.alpha)
model_name += '.h5'

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=example_batch.shape[1:]),
    tf.keras.layers.Conv2D(filters=int(128 * args.alpha), kernel_size=[3, 3], strides=[2, 2],
        use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=int(128 * args.alpha), kernel_size=[3, 3], strides=[1, 1],
            use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=int(128 * args.alpha), kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=len(pr.LABELS)),
    tf.keras.layers.Softmax()
    ])

model_for_pruning = prune_low_magnitude(model, **pruning_params)

Batch Shape: (64, 125, 31, 1)
Data Shape: (125, 31, 1)
Labels: tf.Tensor(
[4 1 3 3 5 3 4 6 4 2 4 4 5 6 3 3 6 3 0 4 5 1 2 1 6 2 3 3 5 0 5 5 4 4 5 2 3
 3 4 5 4 4 4 6 6 2 4 3 0 4 5 5 3 0 6 4 3 5 4 0 6 4 2 1], shape=(64,), dtype=int64)


2023-01-16 12:40:59.537357: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [15]:
# this model uses Transfer Learning... I mean, we transferred a model developed for another course to this course
# model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d   (None, 62, 15, 128)      2306      
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_batch_n  (None, 62, 15, 128)      513       
 ormalization (PruneLowMagni                                     
 tude)                                                           
                                                                 
 prune_low_magnitude_re_lu (  (None, 62, 15, 128)      1         
 PruneLowMagnitude)                                              
                                                                 
 prune_low_magnitude_conv2d_  (None, 62, 15, 128)      294914    
 1 (PruneLowMagnitude)                                           
                                                        

In [16]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)

linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=args.initial_learning_rate,
    end_learning_rate=args.end_learning_rate,
    decay_steps=len(train_ds) * args.epochs,
)
optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
metrics = [tf.metrics.SparseCategoricalAccuracy()]

callbacks = [ tf.keras.callbacks.ModelCheckpoint(filepath=log_dir_tensorboard+model_name,
                                                 save_weights_only=True,
                                                 verbose=1),
             tfmot.sparsity.keras.UpdatePruningStep(), 
             keras.callbacks.TensorBoard(log_dir=log_dir_tensorboard+model_name, histogram_freq=1)]


model_for_pruning.compile(loss=loss, optimizer=optimizer, metrics=metrics)

#if os.path.exists(checkpoint_path):
#    print("Checkpoint found, loading...")
#    model_for_pruning.load_weights(checkpoint_path)
history = model_for_pruning.fit(train_ds, epochs=args.epochs, validation_data=val_ds,callbacks=callbacks,verbose=1) #it was valds


Epoch 1/60
100/101 [============================>.] - ETA: 0s - loss: 1.8421 - sparse_categorical_accuracy: 0.2534
Epoch 1: saving model to ../../datasets/dsl_data/tensorboard_data/run_6_batch_size_64_pruning_initial_step_0.2_initial_learning_rate_0.03_end_learning_rate_0.001_test_percentage_0.2_pruning_initial_step_0.2_initial_sparsity_0.4_alpha_1.h5


OSError: Please specify a non-directory filepath for ModelCheckpoint. Filepath used is an existing directory: ../../datasets/dsl_data/tensorboard_data/run_6_batch_size_64_pruning_initial_step_0.2_initial_learning_rate_0.03_end_learning_rate_0.001_test_percentage_0.2_pruning_initial_step_0.2_initial_sparsity_0.4_alpha_1.h5

In [ ]:
test_loss, test_accuracy = model_for_pruning.evaluate(test_ds)

training_loss = history.history['loss'][-1]
training_accuracy = history.history['sparse_categorical_accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]

print(f'Training Loss: {training_loss:.4f}')
print(f'Training Accuracy: {training_accuracy*100.:.2f}%')
print()
print(f'Validation Loss: {val_loss:.4f}')
print(f'Validation Accuracy: {val_accuracy*100.:.2f}%')
print()
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy*100.:.2f}%')